In [1]:
import pandas as pd
import subprocess

In [2]:
def get_the_table_data():
    while True:
        table_path = input("What is the table path + name? (.../filename.csv):")
        try:
            video_schnitt_df = pd.read_csv(table_path)
            print("Table loaded succesfully")
            return video_schnitt_df
        except Exception as error:
            print("An error occured:", error)
            continue
        break

In [3]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    df = df.astype(str)
    print("Table Columns cleaned")
    return df

In [4]:
def cut_head_tail(original_video,output_file,cut_head="00:00:00",cut_tail="00:50:00"):

    # Construct the command
    command = [
        'ffmpeg',
        '-i', original_video,
        '-ss', cut_head,
        '-to', cut_tail,
        '-c', 'copy',
        output_file
    ]
    
     # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [5]:
def get_video_duration(file_path):
    command = ['ffmpeg', '-i', file_path]
    result = subprocess.run(command, text=True, capture_output=True)
    output_lines = result.stderr.split('\n')
    duration_line = [line for line in output_lines if 'Duration' in line][0]
    duration = duration_line.strip().split(",")[0].split(" ")[1]
    return duration

In [6]:
def get_tail(input_file,output_file,timestamp):

    # -ss = ab dort; -to = bis dort
    command = ['ffmpeg', '-i', input_file, '-ss', timestamp, '-c', 'copy', output_file]

    # Execute the commands
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [7]:
def get_head(input_file,output_file,timestamp):

    # -ss = output ab dort; -to = output bis dort 
    command = ['ffmpeg', '-i', input_file, '-to', timestamp, '-c', 'copy', output_file]

    # Execute the commands
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [8]:
def cut_in_2_pieces(input_file, timestamp, head_output, tail_output):
    
    # -ss = ab dort; -to = bis dort
    head_cmd = ['ffmpeg', '-i', input_file, '-to', timestamp, '-c', 'copy', head_output]
    tail_cmd = ['ffmpeg', '-i', input_file, '-ss', timestamp, '-c', 'copy', tail_output]

# Execute the commands
    try:
        subprocess.run(head_cmd, check=True)
        subprocess.run(tail_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [9]:
def cut_between_pieces(input_file,output_file,cut_head,cut_tail):

    # -ss = ab dort; -to = bis dort
    command = ['ffmpeg', '-i', input_file, '-to', cut_head, '-c', 'copy', head_output]
    command = ['ffmpeg', '-i', input_file, '-to', cut_tail, '-c', 'copy', tail_output]
    
    try:
        subprocess.run(command,check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [10]:
def get_standbild(input_file,output_file,cut_head):
    
    #creat timedelta and add 1 miliseconds
    time_delta = pd.to_timedelta(cut_head)
    new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

    #create dummy for calculation to transform data type back to datetype
    dummy_date = pd.Timestamp('1900-01-01')
    new_timestamp = dummy_date + new_time_delta

    #get the new time back to wished str format
    cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

    # Construct the command
    command = [
        'ffmpeg',
        '-i', input_file,
        '-ss', cut_head,
        '-to', cut_tail,
        '-an', #audio no
        '-c:v', 'copy', output_file
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [11]:
def merge_audio_and_video(video_file,audio_file,output_file):
    #Construct command
    command = ['ffmpeg',
               '-i', video_file,
               '-i', audio_file,
               '-c:v', 'copy',
               '-c:a', 'aac', output_file
               ]
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [12]:
def concatenate_videos(input_file, output_file):
    
    command = [
        'ffmpeg',
        '-f', 'concat', 
        '-i', input_file,
         '-c:v', 'copy',
         '-c:a', 'copy',
        output_file
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print("An error occured:", e)

In [13]:
def get_audio(input_file,output_file,cut_head,cut_tail):
    
    #Construct command
    command = ['ffmpeg',
               '-i', input_file,
               '-ss', cut_head,
               '-to', cut_tail,
               '-vn','-c:a', 'copy', output_file]
    
    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

# Get the file and clean it

In [14]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)

Table loaded succesfully
Table Columns cleaned


# Cut head and tail

In [15]:
# videos need to be in same directory with python script
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        output_file = f"{video_name.split(".")[0]}_ohne_start_ende.mp4"
        cut_head = video_schnitt_df["vorne_abschneiden_bis"][idx]
        cut_tail = video_schnitt_df["hinten_abschneiden_ab"][idx]

        #building logic when just to cut head or tail
        if str(cut_head) == "nan" and str(cut_tail) == "nan":
            continue
        if str(cut_head) == "nan":
            cut_head = "00:00:00"
        if str(cut_tail) == "nan":
            cut_tail = "01:30:00"
    
        print(video_name,cut_head, cut_tail)
        cut_head_tail(video_name, output_file, cut_head, cut_tail)
except Exception as error:
    print("An error occurred:", error)

AD_Streuung.mp4 00:00:08.000 00:03:30.000


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

Chi_Quadrat_Klausuraufgabe.mp4 00:01:40.000 00:05:56.000


frame= 7432 fps=0.0 q=-1.0 Lsize=    7942kB time=00:04:15.98 bitrate= 254.2kbits/s speed= 496x    
video:1673kB audio:6000kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 3.513284%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --

Chi_Quadrat.mp4 00:00:04.200 00:10:59.000


...

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'Empirische_Kovarianz.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf59.27.100
  Duration: 00:11:16.40, start: 0.000000, bitrate: 233 kb/s
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], 32 kb/s, 30 fps, 30 tbr, 15360 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1[0x2](und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 191 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Output #0, mp4, to 'Empirische_Kovarianz_ohne_start_ende.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf60.3.100
  Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt

Empirische_Varianz_Standardabweichung.mp4 00:00:00 00:11:40.800


frame=20776 fps=18403 q=-1.0 Lsize=   21199kB time=00:11:40.77 bitrate= 247.8kbits/s speed= 621x    
video:4026kB audio:16425kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 3.658473%


Erwartungstreue.mp4 00:00:08.000 00:07:10.000


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

# Get Standbild

In [16]:
# videos need to be in same directory with python script
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        output_file = f"{video_name.split(".")[0]}_standbild.mp4"
        cut_head = video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]

        #building logic when just to cut head or tail
        if str(cut_head) == "nan":
            continue
        
        get_standbild(video_name,output_file,cut_head)
except Exception as error:
    print("An error occurred:", error)

# Get audio

In [17]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        
        if str(video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]) == "nan":
            continue
        # timedelta = cut_tail - pd.to_timedelta(cut_head)
        # dummy_date = pd.Timestamp('1900-01-01')
        # new_timestamp = dummy_date + timedelta
        # cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

        #Define arguments
        output_file = f"{video_name.split(".")[0]}_head_audio.mp4"
        cut_head = video_schnitt_df["vorne_abschneiden_bis"][idx]
        cut_tail = video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]

        if str(cut_head) == "nan":
            cut_head = "00:00:00"

        print("video_name",video_name,"output_file", output_file,"cut_head",cut_head,"cut_tail:", cut_tail)
        get_audio(video_name,output_file,cut_head,cut_tail)

except Exception as error:
    print("An error occurred:", error)

# Merge audio and video

In [18]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        if str(video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]) == "nan":
            continue
        video_file = f"{video_name.split(".")[0]}_standbild.mp4"
        audio_file = f"{video_name.split(".")[0]}_head_audio.mp4"
        output_file = f"{video_name.split(".")[0]}_merged_start.mp4"
        
        merge_audio_and_video(video_file,audio_file,output_file)

except Exception as error:
    print("An error occured:", error)

# Concatenate videos to end product

In [19]:
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        #define input for the text file with the videos we want to concat
        video1 = f"{video_name.split(".")[0]}_merged_start.mp4"
        video2 = video_name
        textfile_content = f"file '{video1}'\nfile '{video2}'"

        #create the file with content
        with open('dummy.txt', 'w') as textfile:
            textfile.write(textfile_content)

        #define parameters for concat function
        output_file = f"{video_name.split(".")[0]}_endprodukt_trimmed.mp4"
        textfile_name = "dummy.txt"

        concatenate_videos(textfile_name,output_file)
        
except Exception as error:
    print("An error occured:", error)

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'AD_Streuung_endprodukt_trimmed.mp4']' returned non-zero exit status 1.
An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Chi_Quadrat_Klausuraufgabe_endprodukt_trimmed.mp4']' returned non-zero exit status 1.
An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Chi_Quadrat_endprodukt_trimmed.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Cramers_V_endprodukt_trimmed.mp4']' returned non-zero exit status 1.
An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Effizienz_endprodukt_trimmed.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Empirische_Kovarianz_endprodukt_trimmed.mp4']' returned non-zero exit status 1.
An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Empirische_Varianz_Standardabweichung_Klausuraufgabe_endprodukt_trimmed.mp4']' returned non-zero exit status 1.
An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Empirische_Varianz_Standardabweichung_endprodukt_trimmed.mp4']' returned non-zero exit status 1.
An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Erwartungstreue_endprodukt_trimmed.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

# Remove and Concat

In [20]:
#Get Head
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        #Logik da nicht alle Videos vorher beschnitten sein müssen
        if all(str(video_schnitt_df[col][idx]) == "nan" for col in ["vorne_abschneiden_bis", "hinten_abschneiden_ab", "vorne_bild_durch_standbild_ersetzen_bis"]):
            input_file = video_name
        elif str(video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]) == "nan":
            input_file = f"{video_name.split(".")[0]}_ohne_start_ende.mp4"
        else:
            input_file = f"{video_name.split(".")[0]}_endprodukt_trimmed.mp4"

        timestamp = str(video_schnitt_df["rausschneiden_ab"][idx])
        output_file = f"{video_name.split(".")[0]}_head.mp4"

        get_head(input_file,output_file,timestamp)
        
        continue
except Exception as error:
    print("An error occured:", error)   

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: Command '['ffmpeg', '-i', 'AD_Streuung_ohne_start_ende.mp4', '-to', 'nan', '-c', 'copy', 'AD_Streuung_head.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: Command '['ffmpeg', '-i', 'Chi_Quadrat_Klausuraufgabe_ohne_start_ende.mp4', '-to', 'nan', '-c', 'copy', 'Chi_Quadrat_Klausuraufgabe_head.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

...

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: Command '['ffmpeg', '-i', 'Empirische_Kovarianz_ohne_start_ende.mp4', '-to', 'nan', '-c', 'copy', 'Empirische_Kovarianz_head.mp4']' returned non-zero exit status 1.
An error occurred: Command '['ffmpeg', '-i', 'Empirische_Varianz_Standardabweichung_Klausuraufgabe.mp4', '-to', 'nan', '-c', 'copy', 'Empirische_Varianz_Standardabweichung_Klausuraufgabe_head.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: Command '['ffmpeg', '-i', 'Erwartungstreue_ohne_start_ende.mp4', '-to', 'nan', '-c', 'copy', 'Erwartungstreue_head.mp4']' returned non-zero exit status 1.


frame= 9128 fps=0.0 q=-1.0 Lsize=    9134kB time=00:05:12.49 bitrate= 239.4kbits/s speed= 710x    
video:1479kB audio:7324kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 3.752398%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --

In [21]:
#Get Tail
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        #Logik da nicht alle Videos vorher beschnitten sein müssen
        if all(str(video_schnitt_df[col][idx]) == "nan" for col in ["vorne_abschneiden_bis", "hinten_abschneiden_ab", "vorne_bild_durch_standbild_ersetzen_bis"]):
            input_file = video_name
        elif str(video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]) == "nan":
            input_file = f"{video_name.split(".")[0]}_ohne_start_ende.mp4"
        else:
            input_file = f"{video_name.split(".")[0]}_endprodukt_trimmed.mp4"

        timestamp = video_schnitt_df["rausschneiden_bis"][idx]
        output_file = f"{video_name.split(".")[0]}_tail.mp4"

        get_tail(input_file,output_file,timestamp)
        
        continue
except Exception as error:
    print("An error occured:", error)   

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: Command '['ffmpeg', '-i', 'AD_Streuung_ohne_start_ende.mp4', '-ss', 'nan', '-c', 'copy', 'AD_Streuung_tail.mp4']' returned non-zero exit status 1.
An error occurred: Command '['ffmpeg', '-i', 'Chi_Quadrat_Klausuraufgabe_ohne_start_ende.mp4', '-ss', 'nan', '-c', 'copy', 'Chi_Quadrat_Klausuraufgabe_tail.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: Command '['ffmpeg', '-i', 'Cramers_V_ohne_start_ende.mp4', '-ss', 'nan', '-c', 'copy', 'Cramers_V_tail.mp4']' returned non-zero exit status 1.
An error occurred: Command '['ffmpeg', '-i', 'Effizienz_ohne_start_ende.mp4', '-ss', 'nan', '-c', 'copy', 'Effizienz_tail.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: Command '['ffmpeg', '-i', 'Empirische_Kovarianz_ohne_start_ende.mp4', '-ss', 'nan', '-c', 'copy', 'Empirische_Kovarianz_tail.mp4']' returned non-zero exit status 1.
An error occurred: Command '['ffmpeg', '-i', 'Empirische_Varianz_Standardabweichung_Klausuraufgabe.mp4', '-ss', 'nan', '-c', 'copy', 'Empirische_Varianz_Standardabweichung_Klausuraufgabe_tail.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occurred: Command '['ffmpeg', '-i', 'Erwartungstreue_ohne_start_ende.mp4', '-ss', 'nan', '-c', 'copy', 'Erwartungstreue_tail.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

In [22]:
# Concat head and tail
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():

        #define input for the text file with the videos we want to concat
        video1 = f"{video_name.split(".")[0]}_head.mp4"
        video2 = f"{video_name.split(".")[0]}_tail.mp4"
        textfile_content = f"file '{video1}'\nfile '{video2}'"

        #create the file with content
        with open('dummy.txt', 'w') as textfile:
            textfile.write(textfile_content)

        #define parameters for concat function
        output_file = f"{video_name.split(".")[0]}_endprodukt_remove_and_concat.mp4"
        textfile_name = "dummy.txt"

        concatenate_videos(textfile_name,output_file)
        
except Exception as error:
    print("An error occured:", error)

An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'AD_Streuung_endprodukt_remove_and_concat.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Chi_Quadrat_Klausuraufgabe_endprodukt_remove_and_concat.mp4']' returned non-zero exit status 1.


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fa6cf022d40] Auto-inserting h264_mp4toannexb bitstream filter
Input #0, concat, from 'dummy.txt':
  Duration: N/A, start: 0.000000, bitrate: 233 kb/s
  Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], 41 kb/s, 30 fps, 30 tbr, 15360 tbn
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 192 kb/s
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Output #0, mp4, to 'Chi_Quadrat_endprodukt_remove_and_concat.mp4':
  Metadata:
    encoder         : Lavf60.3.100
  Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt709, progressive), 1920x1080 [SAR 1:1 DAR 16:9], q=2-31, 41 kb/s, 30 fps, 30 tbr, 15360 tbn
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1(und): Audio: a

An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Cramers_V_endprodukt_remove_and_concat.mp4']' returned non-zero exit status 1.
An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Effizienz_endprodukt_remove_and_concat.mp4']' returned non-zero exit status 1.
An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Empirische_Kovarianz_endprodukt_remove_and_concat.mp4']' returned non-zero exit status 1.


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disabl

An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Empirische_Varianz_Standardabweichung_Klausuraufgabe_endprodukt_remove_and_concat.mp4']' returned non-zero exit status 1.


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fc3d9904440] Auto-inserting h264_mp4toannexb bitstream filter


An error occured: Command '['ffmpeg', '-f', 'concat', '-i', 'dummy.txt', '-c:v', 'copy', '-c:a', 'copy', 'Erwartungstreue_endprodukt_remove_and_concat.mp4']' returned non-zero exit status 1.


frame=19654 fps=18497 q=-1.0 Lsize=   20220kB time=00:11:08.37 bitrate= 247.8kbits/s speed= 629x    
video:3848kB audio:15663kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 3.633650%
ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig

# Cut in pieces

In [23]:
for idx, video_name in video_schnitt_df["dateiname"].items():
    cut_time = video_schnitt_df["schnitt_setzen_bei"][idx]
    head_output = f"{video_name}_head_piece.mp4"
    tail_output = f"{video_name}_tail_piece.mp4"
    cut_in_pieces(video_name,cut_time,head_output,tail_output)

NameError: name 'cut_in_pieces' is not defined

In [ ]:
# videos need to be in same directory with python script
for idx, video_name in video_schnitt_df["dateiname"].items():
    output_file = f"{video_name}_standbild.mp4"
    cut_time = video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]

In [ ]:

# Your time string
time_str = "00:00:00.000"

# Convert to Pandas Timedelta
time_delta = pd.to_timedelta(time_str)

# Add one millisecond
new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

# Convert Timedelta to Timestamp for formatting (using a dummy date)
dummy_date = pd.Timestamp('1900-01-01')
new_timestamp = dummy_date + new_time_delta

# Format to "HH:MM:SS.fff"
new_time_str = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

new_time_str


In [ ]:
video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][0]


In [ ]:
duration = pd.to_datetime(get_video_duration("Video.mp4")).strftime('%H:%M:%S.%f')[:-3]
duration = duration + duration

In [ ]:
start_time = pd.to_datetime("00:00:10").strftime('%H:%M:%S.%f')[:-3]
end_time = pd.to_datetime("00:00:15").strftime('%H:%M:%S.%f')[:-3]

In [ ]:
start_time = pd.to_datetime("00:00:10").strftime('%H:%M:%S.%f')[:-3]
end_time = pd.to_datetime("00:00:15").strftime('%H:%M:%S.%f')[:-3]
result = start_time + pd.to_timedelta(end_time) 

In [ ]:
get_video_duration("Video.mp4")